In [2]:
from utils.functions import get_path, search_files, get_filename

from typing import io

import cv2
import plistlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

UsageError: Line magic function `%` not found.


In [3]:
dir_imgs = r'C:\Users\USUARIO\Desktop\Master Data Science\TFM\Code\data\01_CONVERTED\INBreast\FULL'
dir_xml = r'C:\Users\USUARIO\Desktop\Master Data Science\TFM\Code\data\00_RAW\INBreast\AllXML'

def get_inbreast_roi():
    """
    This function loads a osirix xml region as a binary numpy array for INBREAST
    dataset
    @mask_path : Path to the xml file
    @imshape : The shape of the image as an array e.g. [4084, 3328]
    return: numpy array where positions in the roi are assigned a value of 1.
    """

    def load_point(point_string):
        x, y = tuple([int(round(float(num), 0)) for num in point_string.strip('()').split(',')])
        return y, x

    l = []
    for path in search_files(file=dir_xml, ext='xml', in_subdirs=False):
        plist_dict = plistlib.load(open(path, 'rb'), fmt=plistlib.FMT_XML)['Images'][0]
        for roi in plist_dict['ROIs']:
            if roi['Name'] in ['Mass']:
                p = pd.DataFrame(data=[load_point(point) for point in roi['Point_px']], columns=['Y', 'X'])
                l.append([get_filename(path), p.X.max(), p.X.min(), p.Y.max(), p.Y.min(), list(p.values.tolist())])

    df = pd.DataFrame(data=l, columns=['File Name', 'X_MAX', 'X_MIN', 'Y_MAX', 'Y_MIN', 'POINTS'])

    return df

def get_images(p: io):

    l = []
    for path in search_files(file=p, ext='png', in_subdirs=False):
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
        l.append([get_filename(path), img, img.shape[0], img.shape[1]])

    df = pd.DataFrame(data=l, columns=['File Name', 'image', 'height', 'width'])

    return df

In [4]:
df_xml = get_inbreast_roi()
df_imgs = get_images(p=dir_imgs)

In [26]:
# Analisis del tamaño de las imagenes de inbreast. Parece que hay exclusivamente dos tamaños de parche.
df_imgs.apply(lambda x: f'{x.height}, {x.width}', axis=1).value_counts()


4084, 3328    48
3328, 2560    40
dtype: int64

In [62]:
# Analisis de las máscaras
df_xml.loc[:, 'height'] = df_xml.apply(lambda x: x.Y_MAX - x.Y_MIN, axis=1)
df_xml.loc[:, 'width'] = df_xml.apply(lambda x: x.X_MAX - x.X_MIN, axis=1)

df_xml[['height', 'width']].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

,height,width
count,116.000000,116.000000
mean,324.448276,328.060345
std,202.017922,193.817139
min,49.000000,73.000000
10%,126.500000,138.000000
20%,162.000000,166.000000
30%,194.000000,192.500000
40%,223.000000,220.000000
50%,256.500000,281.000000
60%,327.000000,326.000000


In [84]:
df = pd.merge(left=df_imgs, right=df_xml, on=['File Name'], how='left')

In [107]:
def draw_mask(shape: tuple, points: list):

    img = np.zeros(shape=shape)
    cv2.polylines(img, np.int32([np.array(points)]), isClosed=True, color=(255, 255, 255), thickness=5)
    return img

def draw_mask_2(shape: tuple, filename: io):
    from skimage.draw import polygon
    def load_point(point_string):
        x, y = tuple([float(num) for num in point_string.strip('()').split(',')])
        return y, x

    mask = np.zeros(shape)
    plist_dict = plistlib.load(open(get_path(dir_xml, f'{filename}.xml'), 'rb'), fmt=plistlib.FMT_XML)['Images'][0]
    for roi in plist_dict['ROIs']:
        points = [load_point(point) for point in roi['Point_px']]
        x, y = zip(*points)
        x, y = np.array(x), np.array(y)
        poly_x, poly_y = polygon(x, y, shape=shape)
        mask[poly_x, poly_y] = 1
    return mask

In [8]:
imagen = df.iloc[0]

def plot_mask(imagen):

    fig, ax = plt.subplots(1, 4, figsize=(15, 7))

    ax[0].imshow(imagen['image'], cmap='gray')
    ax[1].imshow(cv2.rectangle(imagen['image'], (int(imagen['X_MIN']), int(imagen['Y_MIN'])), (int(imagen['X_MAX']), int(imagen['Y_MAX'])), (255, 255, 255), thickness=-1), cmap='gray')
    ax[2].imshow(draw_mask(shape=(imagen['height'], imagen['width']), points=imagen['POINTS']), cmap='gray')
    ax[3].imshow(draw_mask_2(shape=(imagen['height'], imagen['width']), filename=imagen['File Name']), cmap='gray')

    fig.tight_layout()

NameError: name 'df' is not defined

In [73]:
(imagen['X_MIN'], imagen['Y_MIN']), (imagen['X_MAX'], imagen['Y_MAX'])

((2392.0, 983.0), (2491.0, 1097.0))

In [96]:
img = np.zeros(shape=(df.iloc[0]['height'], df.iloc[0]['width']))
cv2.polylines(img, np.int32([np.array(df.iloc[0].POINTS)]), isClosed=True, color=(255, 255, 255))


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
multiple_regions = df_xml['File Name'].value_counts()
multiple_regions.drop(index=multiple_regions[multiple_regions == 1].index, inplace=True)

df_xml[df_xml['File Name'].isin(multiple_regions.index)]


,File Name,X_MAX,X_MIN,Y_MAX,Y_MIN,POINTS
0,20586908,2491,2392,1097,983,"[[995, 2463], [987, 2453], [983, 2443], [984, ..."
1,20586908,3328,3110,1202,1008,"[[1049, 3196], [1046, 3204], [1043, 3211], [10..."
3,20586960,1889,1798,1397,1284,"[[1306, 1810], [1300, 1814], [1294, 1820], [12..."
4,20586960,2700,2464,965,813,"[[813, 2532], [814, 2539], [816, 2546], [819, ..."
23,22427840,2412,1980,1294,877,"[[1160, 2004], [1171, 2014], [1180, 2024], [11..."
24,22427840,2482,2343,857,691,"[[817, 2389], [814, 2393], [806, 2399], [800, ..."
25,22579730,2061,1647,1372,941,"[[1167, 1647], [1156, 1655], [1144, 1661], [11..."
26,22579730,2243,2102,1018,884,"[[884, 2154], [888, 2145], [890, 2135], [890, ..."
31,22580341,2428,1914,1633,1177,"[[1186, 2224], [1182, 2205], [1180, 2186], [11..."
32,22580341,2223,1830,2408,1943,"[[2113, 1830], [2134, 1830], [2162, 1836], [21..."
